## Coursera Captstone Clustering Course 3 excercises

### click <a href="#task1">here</a> to go to Task 1
### click <a href="#task2">here</a> to go to Task 2
### click <a href="#task3">here</a> to go to Task 3

<a id="task1"></a>

###  Task 1 - web scraping

In [41]:
import requests, csv

In [4]:
import bs4 # beautifulsoup 4 package
from bs4 import BeautifulSoup as bsoup

In [89]:
import pandas as pd

In [27]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [28]:
r = requests.get(url)

In [29]:
html = r.text

In [30]:
soup = bsoup(html,'html.parser')

In [47]:
post_codes_table = soup.find_all("table")[0]

In [90]:
temp_file = "/tmp/zipcodes.csv"
with open(temp_file,"wt") as f:
    w = csv.writer(f)
    
    for row in post_codes_table.select("tbody > tr"):
        row_text = row.get_text(",,,",strip=True).split(",,,")
        w.writerow(row_text)
        # print(row_text)

In [91]:
df = pd.read_csv(temp_file)
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [92]:
df.shape

(288, 3)

In [190]:
df_valid_bor = df[df['Borough'] != "Not assigned"].reset_index(drop=True)

In [191]:
df_valid_bor.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [193]:
grp_post = df_valid_bor.groupby(by='Postcode')

In [194]:
trans = grp_post['Neighbourhood'].transform(lambda x: ",".join(x))

In [195]:
trans.shape

(211,)

In [196]:
df_valid_bor.shape

(211, 3)

In [197]:
df_valid_bor['Neigh2'] = trans
df_valid_bor.head()

,Postcode,Borough,Neighbourhood,Neigh2
0,M3A,North York,Parkwoods,Parkwoods
1,M4A,North York,Victoria Village,Victoria Village
2,M5A,Downtown Toronto,Harbourfront,"Harbourfront,Regent Park"
3,M5A,Downtown Toronto,Regent Park,"Harbourfront,Regent Park"
4,M6A,North York,Lawrence Heights,"Lawrence Heights,Lawrence Manor"


In [198]:
df_valid_bor.drop(columns=["Neighbourhood"], inplace=True)

In [199]:
df_valid_bor.head()

,Postcode,Borough,Neigh2
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M5A,Downtown Toronto,"Harbourfront,Regent Park"
4,M6A,North York,"Lawrence Heights,Lawrence Manor"


In [200]:
df_valid_bor.rename(columns={'Neigh2': 'Neighbourhood'}, inplace=True)

In [201]:
df_valid_bor.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M5A,Downtown Toronto,"Harbourfront,Regent Park"
4,M6A,North York,"Lawrence Heights,Lawrence Manor"


In [202]:
df_valid_bor.drop_duplicates(inplace=True)

In [203]:
df_valid_bor.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
4,M6A,North York,"Lawrence Heights,Lawrence Manor"
6,M7A,Queen's Park,Not assigned


In [204]:
df_valid_bor.reset_index(drop=True, inplace=True)

In [205]:
df_valid_bor.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Not assigned


In [206]:
df_valid_bor.shape

(103, 3)

In [207]:
df_valid_bor['Neighbourhood'] = df_valid_bor.apply(lambda row: row['Borough'] if row['Neighbourhood'] == "Not assigned" else row['Neighbourhood'], axis=1)

In [208]:
df_valid_bor.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [209]:
df_valid_bor.shape

(103, 3)

<a id="task2"></a>

### Task 2 - get latitude and longitude

In [180]:
import geocoder

In [211]:
df2 = df_valid_bor.copy(deep=True)

In [212]:
lats = []
longs = []
for postal_code in df2['Postcode']:

    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
      g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
      lat_lng_coords = g.latlng

    neigh_lat = lat_lng_coords[0]
    neigh_long = lat_lng_coords[1]
    print(lat_lng_coords)
    # df2 = df2.append({'Latitude': neigh_lat, 'Longitude': neigh_long}, ignore_index=False)
    lats.append(neigh_lat)
    longs.append(neigh_long)

[43.752440000000036, -79.32927072599995]
[43.730420577000075, -79.31331999999998]
[43.65512000000007, -79.36263979699999]
[43.72312500000004, -79.45158914699994]
[43.66110229800006, -79.39103499999999]
[43.662242201000026, -79.52837877199994]
[43.811525000000074, -79.19551721399995]
[43.74919500000004, -79.36190541699995]
[43.707535000000064, -79.31177329699995]
[43.65736301100003, -79.37817999999999]
[43.70799000000005, -79.44836733199998]
[43.64969222700006, -79.55394499999994]
[43.78573000000006, -79.15874999999994]
[43.72142500000007, -79.34345278999996]
[43.68966500000005, -79.30716910999996]
[43.65121000000005, -79.37548057699996]
[43.692105179000066, -79.43035499999996]
[43.648573449000025, -79.57824999999997]
[43.76569000000006, -79.17525603599995]
[43.67684518300007, -79.29522499999996]
[43.64516015600003, -79.37367499999993]
[43.68864000000008, -79.45101525299998]
[43.76835912100006, -79.21758999999997]
[43.70976500000006, -79.36390090899994]
[43.65609081300005, -79.384929999

In [213]:
df2.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [214]:
df2['Latitude'] = lats

In [215]:
df2['Longitude'] = longs

In [216]:
df2.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.752440,-79.329271
1,M4A,North York,Victoria Village,43.730421,-79.313320
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.655120,-79.362640
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.723125,-79.451589
4,M7A,Queen's Park,Queen's Park,43.661102,-79.391035


In [217]:
df2.tail()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
98,M8X,Etobicoke,"The Kingsway,Montgomery Road,Old Mill North",43.653690,-79.511117
99,M4Y,Downtown Toronto,Church and Wellesley,43.666585,-79.381302
100,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern,43.648690,-79.385440
101,M8Y,Etobicoke,"Humber Bay,King's Mill Park,Kingsway Park Sout...",43.632765,-79.489601
102,M8Z,Etobicoke,"Kingsway Park South West,Mimico NW,The Queensw...",43.624630,-79.526950


In [219]:
df2.shape

(103, 5)

In [233]:
df2.dtypes

Postcode          object
Borough           object
Neighbourhood     object
Latitude         float64
Longitude        float64
dtype: object

<a id="task3"></a>

### Task 3 - Clustering